create table sales (id_venda varchar primary key, id_prod_caract int, nome_adm varchar, nome_produto varchar, id_funcionario int, nome_funcionario varchar, grupo int, cota int, projeto varchar, nome_cliente varchar, 
					data_venda date, data_venda_abv varchar, data_primeira_parcela date, data_ultima_parcela date, valor_venda numeric, taxa_comissao numeric, valor_comissao numeric, id_income numeric, taxa_receita numeric, 
					valor_receita numeric, data_ultima_parcela_receita date, company_id varchar);
					
create table projects (id_projeto int primary key, nome_projeto varchar, company_id varchar);

create table employees (id_funcionario int primary key, nome_funcionario varchar, cpf varchar, data_adesao date, salario numeric, pix varchar,
						projeto varchar, nome_banco varchar, nr_banco varchar, agencia varchar, conta varchar, titular varchar, documento varchar, emp_validation bool, company_id varchar);
						
create table products (id_produto int primary key, nome_adm varchar, nome_produto varchar, company_id varchar);

create table prod_caract (id_prod_caract int primary key, nome_adm varchar, nome_produto varchar, projeto varchar, taxa_comissao numeric, qtd_parcelas int, taxa_parcelas varchar, company_id varchar);
					   
create table expenses (id_despesa int primary key, tipo_despesa varchar, nome_despesa varchar, qtd int, descricao varchar, valor_salario numeric, valor_comissao numeric, valor_despesa numeric, data_despesa date, 
					  data_despesa_abv varchar, company_id varchar);

create table administrators (id_adm int primary key, nome_adm varchar, company_id varchar);

create table income (id_income int primary key, nome_adm varchar, nome_produto varchar, taxa_comissao numeric, contemplacao numeric, qtd_parcelas int, taxa_parcelas varchar, company_id varchar);

In [14]:
import psycopg2
import pandas as pd
import numpy as np
import streamlit_authenticator as stauth
from datetime import datetime as dt

In [2]:
conn = psycopg2.connect(dbname='postgres', user='postgres', password='1673', host='localhost', port='5432')
cur = conn.cursor()

In [12]:
unix_socket = '/cloudsql/{}'.format("my-project-id:us-central1:my-db-name")

conn = psycopg2.connect(database="app-consorcio", user = "user-db-consorcio", password = "", host = '34.70.179.57')
cur = conn.cursor()

In [13]:
cur.execute('select * from projects;')
lista = list(cur.fetchall())
lista

[(1, 'Externo', 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac'),
 (2, 'Network', 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac'),
 (3, 'Especial 1', 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac'),
 (4, 'Especial 2', 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac'),
 (5, 'Especial 3', 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac'),
 (6, 'Renato', 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac'),
 (7, 'Interno', 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac')]

In [11]:
list_tables = ['projects','employees','products','prod_caract','expenses','administrators','income']
for table in list_tables:
    query = f'SELECT * from {table};'
    cur.execute(query)
    list_col = [x[0] for x in cur.description]
    df = pd.DataFrame(cur.fetchall(), columns=list_col)
    df.to_csv(fr"C:\app-consorcio-data\{table}.csv", index=False)

In [10]:
df[df['grupo'] == 'MCA68']

,id_prod_caract,nome_adm,nome_produto,id_funcionario,nome_funcionario,cota,projeto,nome_cliente,data_venda,data_venda_abv,...,valor_venda,taxa_comissao,valor_comissao,company_id,taxa_receita,valor_receita,id_income,grupo,id_venda,data_ultima_parcela_receita
19,16,BR,BR - MCA,1,Andyara,45,Network,GABRIEL DOS SANTOS PEREIRA FERREIRA,2024-03-05,2024/03,...,30000.0,0.75,225.0,contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac,1.5000000000000002,450.0000000000001,3,MCA68,400517390,2024-06-10


In [7]:
df.columns

Index(['id_prod_caract', 'nome_adm', 'nome_produto', 'id_funcionario',
       'nome_funcionario', 'cota', 'projeto', 'nome_cliente', 'data_venda',
       'data_venda_abv', 'data_primeira_parcela', 'data_ultima_parcela',
       'valor_venda', 'taxa_comissao', 'valor_comissao', 'company_id',
       'taxa_receita', 'valor_receita', 'id_income', 'grupo', 'id_venda',
       'data_ultima_parcela_receita'],
      dtype='object')